# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [90]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [91]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [92]:
data.head()

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20


## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [22]:
sales_by_customer = data.groupby(["CustomerID", "ProductName"]).sum()[["Quantity"]]
sales_by_customer

Quantity
CustomerID ProductName                            
33         Apricots - Dried                      1
           Assorted Desserts                     1
           Bandage - Flexible Neon               1
           Bar Mix - Pina Colada, 355 Ml         1
           Beans - Kidney, Canned                1
...                                            ...
98200      Vol Au Vents                         50
           Wasabi Powder                        25
           Wine - Fume Blanc Fetzer             25
           Wine - Hardys Bankside Shiraz        25
           Yogurt - French Vanilla              25

[63628 rows x 1 columns]

## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [23]:
customer_product = data.pivot_table(values="Quantity", index="CustomerID", columns="ProductName")
customer_product.fillna(0, inplace=True)
customer_product.head()

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
200,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
264,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
356,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
412,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [24]:
similarity_matrix = pd.DataFrame(squareform(pdist(customer_product)), index=customer_product.index, columns=customer_product.index)
# similarity_matrix[33].sort_values().head(6)
similarity_matrix

CustomerID,33,200,264,356,412,464,477,639,649,669,...,97697,97753,97769,97793,97900,97928,98069,98159,98185,98200
CustomerID,,,,,,,,,,,,,,,,,,,,,
33,0.000000,10.723805,9.643651,9.899495,10.440307,10.148892,10.723805,10.246951,10.344080,10.148892,...,206.963765,194.766527,199.145675,183.463893,202.259734,198.075743,203.921554,197.949489,218.824130,203.798921
200,10.723805,0.000000,10.677078,10.723805,11.489125,11.045361,10.770330,10.954451,10.677078,10.488088,...,206.269242,194.285872,198.423789,182.543146,202.044550,197.729613,202.723950,197.349943,218.625250,203.953426
264,9.643651,10.677078,0.000000,10.344080,10.198039,10.488088,10.677078,10.583005,10.488088,10.488088,...,206.366179,194.517351,199.153207,182.789496,201.772149,196.817174,203.315026,197.324606,218.029814,203.806281
356,9.899495,10.723805,10.344080,0.000000,10.630146,10.535654,10.630146,9.949874,10.630146,10.440307,...,206.726873,194.643263,199.025124,182.923481,202.264678,197.575302,204.293906,197.322072,218.142155,203.681123
412,10.440307,11.489125,10.198039,10.630146,0.000000,10.677078,10.677078,10.198039,10.770330,10.392305,...,206.985507,193.889144,199.168271,182.805908,202.034650,197.339808,204.066166,197.466453,217.928887,202.960587
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97928,198.075743,197.729613,196.817174,197.575302,197.339808,197.324606,197.339808,197.441130,197.451260,197.699267,...,266.926956,252.487623,265.753645,246.221445,251.246891,0.000000,252.487623,252.487623,278.388218,276.134025
98069,203.921554,202.723950,203.315026,204.293906,204.066166,204.051464,203.821000,203.919102,203.683578,203.678668,...,273.861279,264.575131,268.095132,263.391344,256.173769,252.487623,0.000000,254.950976,276.134025,264.575131
98159,197.949489,197.349943,197.324606,197.322072,197.466453,197.451260,197.466453,197.694208,197.197870,197.572771,...,266.926956,250.000000,258.602011,256.173769,261.007663,252.487623,254.950976,0.000000,266.926956,266.926956


In [25]:
#arr = np.arange(6).reshape(3,2)
arr = [[0, 1], [2, 3], [4, 5], [1, 1]]
squareform(pdist(arr))
print(pdist(arr))
sqrdf = pd.DataFrame(squareform(pdist(arr)), index=("a","b","c","d"), columns=("a","b","c","d"))
sqrdf["a"].sort_values()

[2.82842712 5.65685425 1.         2.82842712 2.23606798 5.        ]


a    0.000000
d    1.000000
b    2.828427
c    5.656854
Name: a, dtype: float64

In [26]:
test = pd.DataFrame([[1.0,2.0,3.0],[4,5,6],[7,8,9]], columns=["a","b","c"])
test

,a,b,c
0,1.0,2.0,3.0
1,4.0,5.0,6.0
2,7.0,8.0,9.0


In [27]:
pdist(test)

array([ 5.19615242, 10.39230485,  5.19615242])

In [28]:
squareform(pdist(test))

array([[ 0.        ,  5.19615242, 10.39230485],
       [ 5.19615242,  0.        ,  5.19615242],
       [10.39230485,  5.19615242,  0.        ]])

## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [31]:
# 5 most similar registries
similarity_matrix[33].sort_values().head(6)

CustomerID
33      0.000000
3909    9.486833
3531    9.643651
264     9.643651
2503    9.695360
3305    9.746794
Name: 33, dtype: float64

In [46]:
print(customer_product.loc[(customer_product.index == 33) | (customer_product.index == 3909)].T)

CustomerID                         33    3909
ProductName                                  
Anchovy Paste - 56 G Tube           0.0   0.0
Appetizer - Mini Egg Roll, Shrimp   0.0   0.0
Appetizer - Mushroom Tart           0.0   0.0
Appetizer - Sausage Rolls           0.0   0.0
Apricots - Dried                    1.0   0.0
...                                 ...   ...
Yeast Dry - Fermipan                0.0   0.0
Yoghurt Tubes                       0.0   0.0
Yogurt - Blueberry, 175 Gr          0.0   0.0
Yogurt - French Vanilla             1.0   0.0
Zucchini - Yellow                   0.0   0.0

[452 rows x 2 columns]


## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [80]:
filter_list = [33, 3909, 3531, 264, 2503, 3305]
top5to33 = data[data["CustomerID"].isin(filter_list)].sort_values(by="CustomerID")
top5to33

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity
44889,33,Lindsay,Santana,2448019,340,Veal - Sweetbread,1
33913,33,Lindsay,Santana,1943474,306,"Pepper - White, Ground",1
34816,33,Lindsay,Santana,2220112,289,Sword Pick Asst,1
35917,33,Lindsay,Santana,4835339,328,Tuna - Salad Premix,1
36469,33,Lindsay,Santana,255580,317,Water - Green Tea Refresher,1
...,...,...,...,...,...,...,...
11434,3909,Keri,Stafford,678119,350,Isomalt,1
35691,3909,Keri,Stafford,194233,290,Sauce - Rosee,1
11132,3909,Keri,Stafford,1892174,241,Cheese - Cheddarsliced,1
39074,3909,Keri,Stafford,288007,396,Tea - Jasmin Green,1


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [86]:
grouped = top5to33.groupby("ProductName").sum().sort_values(by="Quantity", ascending=False)
grouped.head()

,CustomerID,SalesID,ProductID,Quantity
ProductName,,,,
"Salsify, Organic",13162,13788247,2610,6
Lettuce - Spring Mix,3861,12222877,1772,4
Quiche Assorted,7737,11817279,928,4
Wine - Crozes Hermitage E.,5841,9238155,153,3
"Lamb - Pieces, Diced",6445,11468395,1251,3


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [89]:
recom = pd.merge(top5to33, grouped, how="left", on="ProductID")
recom.loc[recom["CustomerID_x"] == 33]

,CustomerID_x,FirstName,LastName,SalesID_x,ProductID,ProductName,Quantity_x,CustomerID_y,SalesID_y,Quantity_y
0,33,Lindsay,Santana,2448019,340,Veal - Sweetbread,1,NaN,NaN,NaN
1,33,Lindsay,Santana,1943474,306,"Pepper - White, Ground",1,NaN,NaN,NaN
2,33,Lindsay,Santana,2220112,289,Sword Pick Asst,1,NaN,NaN,NaN
3,33,Lindsay,Santana,4835339,328,Tuna - Salad Premix,1,7440.0,2398856.0,2.0
4,33,Lindsay,Santana,255580,317,Water - Green Tea Refresher,1,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...
60,33,Lindsay,Santana,1779080,245,Grouper - Fresh,1,NaN,NaN,NaN
61,33,Lindsay,Santana,2826875,181,Phyllo Dough,1,33.0,2826875.0,1.0
62,33,Lindsay,Santana,1465031,320,Yogurt - French Vanilla,1,NaN,NaN,NaN
63,33,Lindsay,Santana,1899667,432,Beer - Original Organic Lager,1,NaN,NaN,NaN


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.